In [2]:
#import itertools
#import pandas as pd
#import sqlalchemy as sqa
import numpy as np
from tensorflow.keras import models, layers, metrics, optimizers, utils
from tensorflow.keras.models import model_from_json, Sequential
from tensorflow.keras.layers import Input, Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras import backend as K
import tensorflow as tf
#import datetime

/Users/stefanroth/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/stefanroth/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/stefanroth/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/stefanroth/.local/lib/python3.7/site-packages/tensorflow/python/frame

In [6]:
max_features = 53
max_len = 7
vector_size = 4

In [9]:
# Card embeddings
card_input = Input(shape=(7,))
card_output = layers.Embedding(max_features, vector_size, input_length=max_len)(card_input)
#card_output = layers.BatchNormalization()(card_output)
card_output = layers.Flatten()(card_output)

# Other information
state_input = Input(shape=(13,))

#Merge and add dense layer
merge_layer = layers.concatenate([state_input, card_output])
x = layers.BatchNormalization()(merge_layer)
x = layers.Dense(64, activation='relu')(x)
#x = layers.Dense(64, activation='relu')(merge_layer)
x = layers.BatchNormalization()(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.BatchNormalization()(x)
main_output = layers.Dense(3, activation='softmax')(x)

# Define model with two inputs
model = models.Model(inputs=[card_input, state_input], outputs=[main_output])


#    action_prob_ph = self.model.output
action_prob_ph = model.output#K.sum(self.model.output)
#print(f'The probabilities for the three actions are {action_prob_ph}')
action_onehot_ph = K.placeholder(shape=(None, 3),
                                         name="action_onehot")
reward_ph = K.placeholder(shape=(None,),
                                            name="reward")

action_prob = K.sum(action_prob_ph * action_onehot_ph, axis=1)
log_action_prob = K.log(action_prob)

loss = - log_action_prob * reward_ph
loss = K.mean(loss)

adam = optimizers.Adam(clipvalue=0.5)

updates = adam.get_updates(params=model.trainable_weights,
                           #constraints=[],
                           loss=loss)

train_fn = K.function(inputs=[model.input[0],
                                    model.input[1],
                                   action_onehot_ph,
                                   reward_ph],
                           outputs=[loss, action_onehot_ph+1, action_prob_ph, action_prob, log_action_prob, reward_ph+1], updates=updates)

model.compile(optimizer=adam, loss='categorical_crossentropy')

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 7)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 7, 4)         212         input_7[0][0]                    
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 13)           0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 28)           0           embedding_1[0][0]                
__________________________________________________________________________________________________
concatenat

In [13]:
model.output()

<tf.Tensor 'dense_5/Softmax:0' shape=(?, 3) dtype=float32>